In [17]:
# Import some useful libraries
%matplotlib inline
import pandas as pd
import urllib
import xml.etree.ElementTree as ET
import numpy as np
import nltk
import rake
from bs4 import BeautifulSoup as BSoup
import nlp_helpers
import gensim
import cleaner
from cleaner import Cleaner
from classifier import *
%load_ext autoreload
%autoreload 2

# display all pandas columns
pd.set_option('display.max_columns', 100)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# NLP content extraction with GENSIM

First attempt of transcript content extraction
(Tokenization|Parser|Text Segmentation|Word Break rules|Text Analysis)

http://gerardnico.com/wiki/natural_language/parser


**Tested: **
* RAKE: good but basic, needs a lot of by-hand process (stemming, ...) but perfectly controlled so we can control everything
* TextRank: in python2... too lazy to change my path ^^
* KEA: Do not seem to be still maintained 
* MAUI: Improvement of KEA, works with supervised learning and controlled vocabulary. Works with java

## Gensim

https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

* NLTK, a natural language toolkit for Python. A useful package for any natural language processing.
    * For Mac/Unix with pip: $ sudo pip install -U nltk.
    
* stop_words, a Python package containing stop words.
    * For Mac/Unix with pip: $ sudo pip install stop-words.
    
* gensim, a topic modeling package containing our LDA model.
    * For Mac/Unix with pip: $ sudo pip install gensim.
    
* clips.pattern for gensim lemmatization
    * -_- only python2 !! If really usefull, I'll do some trick to adapt it

### CSV load

In [4]:
df_transcripts = pd.read_csv("../scraping/data/Transcript.csv", nrows=5000)
df_transcripts = df_transcripts.loc[df_transcripts['LanguageOfText'] == 'FR']
df_transcripts['Text'] = df_transcripts['Text'].apply(cleaner.clean_html)
print(df_transcripts.shape)
df_transcripts.head(2)

(925, 29)


,Unnamed: 0,CantonAbbreviation,CantonId,CantonName,CouncilId,CouncilName,DisplaySpeaker,End,Function,ID,IdSession,IdSubject,Language,LanguageOfText,MeetingCouncilAbbreviation,MeetingDate,MeetingVerbalixOid,Modified,ParlGroupAbbreviation,ParlGroupName,PersonNumber,SortOrder,SpeakerFirstName,SpeakerFullName,SpeakerLastName,Start,Text,Type,VoteId
0,0,VD,22.0,Vaud,1.0,Conseil national,True,1999-12-06T14:50:21.663,NaN,1,4601,1,FR,FR,N,19991206,637,2015-11-09T11:48:16.91,C,Groupe démocrate-chrétien,498.0,3,Jacques,Neirynck Jacques,Neirynck,1999-12-06T14:36:50.007,La séance d'aujourd'hui s'ouvre sous le signe ...,1,NaN
2,2,VD,22.0,Vaud,1.0,Conseil national,True,1999-12-06T14:33:39.803,NaN,3,4601,1,FR,FR,N,19991206,637,2015-11-09T11:48:16.937,C,Groupe démocrate-chrétien,498.0,1,Jacques,Neirynck Jacques,Neirynck,1999-12-06T14:32:24.303,"Madame la Présidente de la Confédération, Seh...",1,NaN


### Clean Up pipeline

* HTML tags
* lower cases
* tokenization
* removing stop words
* stemming

In [9]:
cleaner = Cleaner()
text = df_transcripts['Text']
clean_text = cleaner.cleaning_pipeline_series(text)
clean_text.head()

0     [séanc, aujourd, hui, ouvr, sign, hasard, dont...
2     [madam, président, confédérat, geehrt, frau, b...
11    [sembl, acqui, majorité, conseil, nation, élir...
12    [ainsi, rotat, immuabl, présidenc, voudrait, a...
13    [venez, recevoir, feuill, vert, donn, list, gr...
Name: Text, dtype: object

In [18]:
classifier = Classifier()
classifier.define_dictionary(clean_text)

[INFO] Dictionary defined


In [19]:
classifier.define_model(clean_text, 4, 1)

[INFO] LDA model defined
[INFO] define_model took 13.629 s


In [20]:
classifier.print_model()

Cluster  0 :  (0, '0.013*"fédéral" + 0.013*"conseil" + 0.009*"commiss" + 0.009*"plu" + 0.008*"suiss" + 0.006*"loi" + 0.005*"proposit" + 0.005*"projet" + 0.004*"politiqu" + 0.004*"articl"')
Cluster  1 :  (1, '0.014*"conseil" + 0.011*"fédéral" + 0.010*"commiss" + 0.009*"plu" + 0.006*"franc" + 0.006*"suiss" + 0.004*"pay" + 0.004*"certain" + 0.004*"proposit" + 0.004*"assur"')
Cluster  2 :  (2, '0.011*"conseil" + 0.010*"fédéral" + 0.009*"suiss" + 0.009*"plu" + 0.005*"commiss" + 0.004*"politiqu" + 0.004*"proposit" + 0.004*"projet" + 0.004*"franc" + 0.004*"assur"')
Cluster  3 :  (3, '0.010*"plu" + 0.009*"conseil" + 0.008*"commiss" + 0.007*"suiss" + 0.006*"loi" + 0.006*"fédéral" + 0.005*"pay" + 0.005*"proposit" + 0.005*"projet" + 0.004*"politiqu"')


In [21]:
test_text = df_transcripts['Text'][0]
test_text

'La séance d\'aujourd\'hui s\'ouvre sous le signe de deux hasards dont la conjonction est tout à fait remarquable. Un pur hasard du calendrier veut que nous nous réunissions sept ans, jour pour jour, après que le peuple suisse ait décidé de ne pas adhérer à l\'Espace économique européen. Sept ans plus tard, la campagne électorale fut centrée sur le même problème, qui n\'a manifestement pas reçu une solution acceptable pour tous, tant l\'opinion semble partagée à cet égard. Un second hasard, d\'ordre purement biologique, fait que le doyen d\'âge de cette Assemblée, président provisoire de cette séance, se trouve être un Suisse naturalisé de fraîche date. Par une ironie supplémentaire, il est même natif de Bruxelles. (Applaudissements) Existe-t-il un autre pays que la Suisse où le Parlement est présidé par un travailleur immigré? La coïncidence de ces deux hasards démontre à la fois que la Suisse n\'est pas xénophobe, comme l\'ont laissé entendre certains commentaires superficiels, mais 

In [22]:
classifier.classify(test_text)

[(2, 0.38913372809247826), (3, 0.61002106626090036)]